## Compress the Base LLM using LLM Compressor:
Now that we have established **accuracy and performance baselines** for the base model (`Llama-3.1-8B-Instruct`), the next step is to apply **model compression**. Specifically, we will quantize the base model and later evaluate the compressed version for both accuracy and system-level performance.

This will allow us to assess whether model compression improves deployment metrics—such as **Time to First Token (TTFT)** while maintaining accuracy comparable to the base model.


This step focuses on **reducing the model’s memory footprint** through quantization, resulting in a smaller and more deployment-efficient model.

**Note**: We use **data-aware quantization**, which relies on representative calibration data to preserve model quality.

**Goal**: Reduce model size (e.g., FP16 → INT8 / INT4) while retaining accuracy and inference performance.

**Key Actions**:

- Load the base model.

- Measure its size and memory usage.

- Use a calibration dataset (e.g., WikiText, UltraChat) to collect activation statistics.

- Apply a quantization recipe (e.g., SmoothQuant + GPTQ modifier).

- Save the compressed model and verify size reduction.

**Outcome**:

- Compressed model saved on disk.

- Model size reduced, typically by 50% (depending on quantization scheme).

### Install Dependencies

In [2]:
!pip install .
!pip install torch==2.9.0

Running the above cell might return errors like `ERROR: pip's dependency resolver does not currently take into account... llmcompressor 0.8.1 requires torch<=2.8.0,>=2.7.0, but you have torch 2.9.0 which is incompatible.` which can be safely ignored. 

In [1]:
import torch
from datasets import load_dataset
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from transformers import AutoModelForCausalLM, AutoTokenizer
from utils import model_size_gb, tokenize_for_calibration

/opt/app-root/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# check available device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Loading Base Model

**Make sure to kill any running processes using `nvidia-smi` and `kill -9 <pid>` that might be consuming GPU memory.**

While loading the model using **from_pretrained** using transformers' **AutoModelForCausalLM** class, we specify the data type using the **torch_dtype** parameter and set it to **auto** so the model is loaded in the data type specified in its config.
Otherwise, PyTorch loads the weights in **full precision (fp32)**.

In [3]:
# set up variables
base_model_path = "../base_model"
compressed_model_path = "../Llama_3.1_8B_Instruct_int8_dynamic"

In [4]:
# loading model and tokenizer from huggingfaceabs
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForCausalLM.from_pretrained(
    base_model_path, torch_dtype="auto", device_map="auto"
)
print("Base model loaded")

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.50it/s]

Base model loaded


In [5]:
# check model size
# !du -sh {base_model_path}
model_size = model_size_gb(model)
print(f"The size of the base model is: {model_size:.4f}GB")

The size of the base model is: 14.9575GB


### Preparing Calibration Dataset

Since we are using data-aware quantization to compress the base model, we need a dataset to calibrate the model with real or representative inputs. For the sake of this example, we will use a small, general-purpose dataset for faster processing. Specifically, we use the `wikitext-2-raw-v1` version of the WikiText dataset which is the smaller version.  More information on why to use a calibration dataset is provided in [Compression.md](../docs/Compression.md)

In [6]:
# Define the dataset to use for calibration
dataset_id = "wikitext"

# Specify the configuration / version of the dataset
config = "wikitext-2-raw-v1"  # Small version (~2M tokens), raw text format

# Set the number of calibration samples based on available device
# - On GPU: use more samples to get more accurate activation statistics
# - On CPU: reduce samples to prevent memory issues and keep demo fast
num_calibration_samples = 512 if device == "cuda" else 16

# Set the maximum sequence length for calibration
max_sequence_length = 1024 if device == "cuda" else 16

# Load the dataset using Hugging Face Datasets API
# This downloads train split of the dataset
ds = load_dataset(dataset_id, config, split="train")
# Shuffle and grab only the number of samples we need
ds = ds.shuffle(seed=42).select(range(num_calibration_samples))

In [7]:
# inspect the dataset
print(f"columns in the {dataset_id}: {ds.column_names}\n")
print(ds[0])

columns in the wikitext: ['text']

{'text': ' Continuous , short @-@ arc , high pressure xenon arc lamps have a color temperature closely approximating noon sunlight and are used in solar simulators . That is , the chromaticity of these lamps closely approximates a heated black body radiator that has a temperature close to that observed from the Sun . After they were first introduced during the 1940s , these lamps began replacing the shorter @-@ lived carbon arc lamps in movie projectors . They are employed in typical 35mm , IMAX and the new digital projectors film projection systems , automotive HID headlights , high @-@ end " tactical " flashlights and other specialized uses . These arc lamps are an excellent source of short wavelength ultraviolet radiation and they have intense emissions in the near infrared , which is used in some night vision systems . \n'}


**Datset inspection shows the we need to extract column ```text``` and pass it as input to the model.**

### When to Use a Custom Template for Calibration

Use a **custom template** when you want the calibration text to closely mimic the input format your model will see in production.  

For example, if your model is **instruction-following** or **chat-based**, providing the template the model was originally trained on or the template that will be used during inference ensures that the activation statistics collected during calibration reflect realistic usage patterns. 

This can improve the accuracy of quantization and compression.

If your model can handle raw text and doesn’t require a specific format, you can rely on the default template instead.

A custom template can be provided to the `tokenize_for_calibration` function using the `custom_template` argument. It accepts the following format:

```python
custom_template = {
 "template_text": "Instruction: {content}\nOutput:", 
 "placeholder": "content"
}

In [8]:
# to get activations for the calibration dataset, we need to:
# 1. extract the samples from the dataset
# 2. tokenize samples in the dataset
input_column = "text"

# Call tokenize_for_calibration using dataset.map
tokenized_dataset = ds.map(
    lambda batch: tokenize_for_calibration(
        examples=batch,  # batch from Hugging Face dataset
        input_column=input_column,  # the column containing text to calibrate
        tokenizer=tokenizer,  # your Hugging Face tokenizer
        max_length=max_sequence_length,  # maximum sequence length
        model_type="chat",  # use chat template if no custom template
        custom_template=None,  # optional, provide a dict if you want a custom template
    ),
    batched=True,
)

In [9]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 512
})

### Quantizing/Compressing Base Model to INT8
After preparing the dataset for calibration, we define a recipe for quantization. For quantization scheme `W8A8-INT8`, we use `SmoothQuantModifier` followed by `GPTQModifier`.

More details on what SmoothQUant and GPTQ algorithms are provided in [Compression.md](Compression.md).

In [10]:
# Define the quantization scheme
scheme = "W8A8"  # W8A8 means 8-bit weights and 8-bit activations

# Strength for SmoothQuant smoothing
# This controls how much the activation values are smoothed to reduce outliers
smoothing_strength = 0.8

# Create SmoothQuant modifier
# - smooths activations before quantization to improve stability and reduce degradation
smooth_quant = SmoothQuantModifier(smoothing_strength=smoothing_strength)

# Create GPTQ modifier
# - targets="Linear" quantizes only Linear layers (e.g., feedforward layers)
# - scheme=scheme uses the W8A8 quantization scheme
# - ignore=["lm_head"] preserves the LM head to avoid generation quality loss
quantizer = GPTQModifier(targets="Linear", scheme=scheme, ignore=["lm_head"])

# Combine the modifiers into a recipe list
# The order matters: first apply SmoothQuant, then GPTQ
recipe = [smooth_quant, quantizer]

# Perform quantization
oneshot(
    model=base_model_path,  # Model to quantize
    dataset=tokenized_dataset,  # Calibration dataset, used for both SmoothQuant & GPTQ
    recipe=recipe,  # List of quantization modifiers to apply
    output_dir=compressed_model_path,  # Directory to save the quantized model
    max_seq_length=2048,  # Maximum sequence length for calibration
    num_calibration_samples=512,  # Number of samples used for calibration
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 127.41it/s]


2026-01-13T14:11:50.033633+0000 | reset | INFO - Compression lifecycle reset
2026-01-13T14:11:50.043707+0000 | _create_default_logger | INFO - Logging all LLM Compressor modifier-level logs to sparse_logs/13-01-2026_14.11.50.log
2026-01-13T14:11:50.044993+0000 | from_modifiers | INFO - Creating recipe from modifiers
2026-01-13T14:11:50.045455+0000 | _infer_mappings_from_model | INFO - No SmoothQuantModifier.mappings provided, inferring from model...
2026-01-13T14:11:50.815363+0000 | initialize | INFO - Compression lifecycle initialized for 2 modifiers
2026-01-13T14:11:50.816990+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `SmoothQuantModifier`


(1/33): Calibrating: 100%|██████████| 512/512 [00:03<00:00, 146.55it/s]

2026-01-13T14:11:55.526979+0000 | _apply_smoothing | INFO - Smoothing with model.layers.0.input_layernorm
2026-01-13T14:11:55.533888+0000 | _apply_smoothing | INFO - Smoothing with model.layers.0.post_attention_layernorm



(2/33): Calibrating: 100%|██████████| 512/512 [00:02<00:00, 249.60it/s]

...

2026-01-13T14:29:37.597004+0000 | compress | METRIC - time 1.51s
2026-01-13T14:29:37.597874+0000 | compress | METRIC - error 28.22
2026-01-13T14:29:37.598446+0000 | compress | METRIC - GPU 0 | usage: 41.61% | total memory: 48 GB
2026-01-13T14:29:37.598823+0000 | compress | METRIC - Compressed module size: 33.56672 MB
2026-01-13T14:29:37.599456+0000 | compress_modules | INFO - Quantizing model.layers.31.self_attn.k_proj using 512 samples
2026-01-13T14:29:39.113656+0000 | compress | METRIC - time 1.51s
2026-01-13T14:29:39.114529+0000 | compress | METRIC - error 19.26
2026-01-13T14:29:39.115143+0000 | compress | METRIC - GPU 0 | usage: 41.62% | total memory: 48 GB
2026-01-13T14:29:39.115507+0000 | compress | METRIC - Compressed module size: 8.39168 MB
2026-01-13T14:29:39.116154+0000 | compress_modules | INFO - Quantizing model.layers.31.self_attn.v_proj using 512 samples
2026-01-13T14:29:40.620702+0000 | compress | METRIC - time 1.50s
2026-01-13T14:29:40.621569+0000 | compress | METRIC - 

(33/33): Propagating: 100%|██████████| 512/512 [00:03<00:00, 157.76it/s]


2026-01-13T14:30:01.210386+0000 | finalize | INFO - Compression lifecycle finalized for 2 modifiers
2026-01-13T14:30:01.255584+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 224it [00:20, 10.78it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

### Checking model size

In [11]:
# Load quantized model
model_quant = AutoModelForCausalLM.from_pretrained(compressed_model_path)
model_quant.config.dtype = model.config.torch_dtype
model_quant.save_pretrained(compressed_model_path)
model_size = model_size_gb(model_quant)
print(f"Model size (GB): {model_size}")

Compressing model: 224it [00:00, 1292.67it/s]
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.45it/s]
`torch_dtype` is deprecated! Use `dtype` instead!


Model size (GB): 8.460090637207031


### Observation
After quantizing the model, the size has clearly reduced from 14.9GB to 8GB. 


**ALTERNATIVELY**, llm-compressor also supports FP8 quantization. This conversion does not require any calibration dataset. Uncomment the below cell to quantize the model to FP8.

In [ ]:
# recipe = QuantizationModifier(
#   targets="Linear", scheme="FP8_DYNAMIC", ignore=["lm_head"])

# oneshot(model=model_name, recipe=recipe, output_dir=compressed_model_path)

# # Load quantized model
# model_quant = AutoModelForCausalLM.from_pretrained(compressed_model_path)
# model_quant.config.dtype = model.config.torch_dtype
# model_quant.save_pretrained(compressed_model_path)
# model_size = model_size_gb(model_quant)
# print(f"Model size (GB): {model_size}")

Now that we have reduced the model size, the next step is to evaluate this compressed model to make sure the accuracy has retained after compression. This is done in step `04_Compressed_Accuracy_Benchmarking`.